# Topic Modelling with LDA

#### Acknowledge: The exercise is adopted from Intel AI Developer Program


#### For illustration only

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
documents = ["cat eat rice", "secret message", "today go shopping"]
tf_vectorizer = CountVectorizer()
tf_vectorized_documents = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 10
lda_model = LatentDirichletAllocation(n_components=no_topics)
lda_output = lda_model.fit_transform(tf_vectorized_documents)
lda_components = lda_model.components_


###  Import the neccessary libraries
- LDA works only with bag of words approach only
- Regular expressions re, gensim and spacy are used to process texts. 
- PyLDAvis and matplotlib for visualization and numpy
- Pandas for manipulating and viewing data in tabular format.



In [2]:
# Sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

# 
import numpy as np
import pandas as pd
import re, nltk


# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\David\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


#### display_topic() is a commonly used function to display topics and related terms
- model - the lda model
- feature_names - the features names
- no_top_words - how many terms to display

** Do not change this function ** 

In [3]:

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

#### Function for Data Cleaning
- Clean_documents() is a function to perform data cleansing for a raw document
- This is a snub function for now. When you prepare your own set of data, you will write your own pre-processing logic

In [4]:
def clean_documents(document):
    # placeholder: Write data preparation codes here
    
    return document

###  Data Processing 

#### This is the section to modify if you have other sources
- Load in the documents from its source
- The LDA topic model algorithm requires a document word matrix as the main input.
- Vectorise the document using count vectorizing
- LDA can only use raw term counts for LDA because it is a probabilistic graphical model


In [5]:

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = clean_documents(dataset.data)

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf_vectorized_documents = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

#### Create the LDA model and apply LDA to the corpus of document
- Fit and transform the vectorize document (tf)
- Display the model attributes
- Displays the topics terms
- Since most cells contain zeros, the result (lda_model) will be in the form of a sparse matrix to save memory


In [6]:

no_topics = 20    # this s just a wild guess
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
lda_output = lda_model.fit_transform(tf_vectorized_documents)

print(lda_model)  # To look at the modelModel attributes


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


### Preview the terms of each topic
- Displays the topics terms

In [7]:
no_top_words = 8
display_topics(lda_model, tf_feature_names, no_top_words)

Topic 0:
don think db use like just want good
Topic 1:
mr gun law guns control stephanopoulos crime laws
Topic 2:
ve like line post bike thanks got right
Topic 3:
10 11 25 15 12 17 14 16
Topic 4:
game team games play season hockey league players
Topic 5:
does people question problem case point answer rules
Topic 6:
windows thanks card does dos window use know
Topic 7:
new 1993 university april national 000 health information
Topic 8:
just don say think know way did like
Topic 9:
key car used output number bit use chip
Topic 10:
edu file com available program ftp information files
Topic 11:
god jesus people believe christian bible life church
Topic 12:
drive scsi disk hard mac apple drives controller
Topic 13:
people armenian israel armenians war jews turkish israeli
Topic 14:
space nasa science earth launch data water satellite
Topic 15:
ax max b8f g9v a86 pl 145 1d9
Topic 16:
year years ago air problem 000 weeks months
Topic 17:
government president encryption public people security l

### Finding dominate topics for a document
- To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.


In [9]:

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(documents))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .2 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style to first 25 document
df_document_topic2 = df_document_topic.head(25).style.applymap(color_green).applymap(make_bold)
df_document_topic2



,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0,0,0,0,0,0,0,0.35,0.37,0,0,0,0,0.26,0,0,0,0,0,0,8
Doc1,0,0,0.16,0,0.21,0,0,0,0.23,0,0,0.33,0.04,0,0,0,0,0,0,0,11
Doc2,0.18,0,0.08,0,0,0,0,0.08,0.3,0,0,0,0,0.33,0,0,0,0,0,0,13
Doc3,0,0,0.17,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0.08,0.39,0,0.22,17
Doc4,0,0,0.65,0,0.08,0.23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
Doc5,0,0,0,0,0,0,0,0,0,0,0,0.34,0,0,0,0,0,0,0,0.58,19
Doc6,0,0.16,0,0.11,0.1,0,0.31,0,0.02,0,0,0,0,0,0,0,0,0,0.06,0.22,6
Doc7,0,0,0,0.03,0,0,0,0.04,0.79,0,0,0,0,0.08,0,0,0.03,0,0,0,8
Doc8,0.85,0,0,0,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0,0,0,0
Doc9,0,0,0.16,0,0,0,0,0,0.1,0.37,0,0,0,0,0,0,0.07,0,0.05,0.24,9


### Question: Inspect the content of the documents

- Visually look for a document with a dominant topic > 0.6 ( doc 14/topic 6 , doc 18/topic 3)
- Use the document number to access the original corpus
- prints the related document / text

In [12]:
documents[13]


'Hi Damon,  No matter what system or explanation of creation you wish\nto accept, you always have to start with one of two premises, creation\nfrom nothing, or creation from something.  There are no other\nalternatives.  And if we accept one or the other of those two\npremises, then again there are two alternatives, either creation was\nrandom, or was according to some plan.\n   If it was random, I am unable to accept that the complex nature of\nour world with interrelated interdependent organisms and creatures\ncould exist as they do.  Therefore I am left with creation under the\ncontrol of an intelligence capable of devising such a scheme.  I call\nthat intelligence God.\n   I also prefer the "Creatio ex nihilo" rather than from chaos, as it\nis cleaner.\n   There is obviously no way to prove either or neither.  We are and\nwe must have come from somewhere.  Choose whatever explanation you\nfeel most comfortable with, Damon.  You are the one who has to live\nwith your choice.\nShalom

### Optional Question:
- If we want the table above to assign a different color (e.g. red) to documents with topics weight of 0.5, what code changes are needed?


### Advance Question:
* Having topics named as topic0, topic1 etc isn't helpful in a real application. Discuss several possible ways to assign meaningful names to the topics. Then, from the topics list, to retrieve the top 10 documents with that dominant topic.
    

#### Reference:
- www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/
